In [43]:
import time
import PICclass as PIC
import numpy as np
from matplotlib import rc, rcParams
import matplotlib.pyplot as plt
import random

rc('xtick', labelsize=22)
rc('ytick', labelsize=22)
rcParams['lines.linewidth'] = 4
rcParams['font.size'] = 35
clist = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#7f7f7f', '#e377c2', '#8c564b', '#17becf', '#bcbd22']

###############################################################################
### Data ###
"""Here are all the stuff to declare before running the script. It consists in the name of the data files to be analyzed but also
in the details of the simulation box as well as how you want to analyze them """

#C> - Name of the simulation where to find the MHD data
#C>---------------------------------------------------- 
MHD_prefix = "PIC_SHOCK_GAS_test2"
MHD_workdir = "/travail/wyao/data/" + MHD_prefix + "/"

#C> - Name of the simulation
#C>-------------------------
prefix = "PIC_SHOCK_GAS_test2"
nickname = "GAS_2e-2"
print "Studying simulation", prefix

#C> - Path to get the data
#C>-----------------------
workdir = "/travail/wyao/data/" + prefix + "/PIC_reduced/"

#C> - Path to save the figures
#C>---------------------------
savepath = "/travail/wyao/data/" + prefix + "/Treatment/"

###############################################################################

#C> - Required arguments to pass to the PICclass script
#C>----------------------------------------------------
Z = 1.
atomic_weight = 1.

#C> - Informations about times
#C>---------------------------
MHD_Time_start = 0.0
MHD_Time_end = 20.0
MHD_time_out_frequency = 2.0

PIC_Time_start = 0.0                                                        #Time when the PIC module was activated in the simulation; nanosecond unit
PIC_Time_end = 20.0 #17.2                                                    #Time when the PIC module wrote its last output files; nanosecond unit.
PIC_time_out_frequency = 2.0

PIC_iter_start = 0
PIC_iter_stop = int(PIC_Time_end/PIC_time_out_frequency)

pic = PIC.PIC(workdir, prefix, 0, savepath, Z, atomic_weight)
elementary_charge, gamma, K_Boltzmann, proton_mass, electron_mass, c_light, mu_zero = pic.load_cst()

Studying simulation PIC_SHOCK_GAS_test2


In [47]:
timing_start = time.time()

pic = PIC.PIC(workdir, prefix, 0, savepath, Z, atomic_weight)

pic.SecurityCheckPaths("")

ipx,ipy,ipz, \
iPx,iPy,iPz, \
iBx,iBy,iBz, \
iEx,iEy,iEz, \
imass,icharge,itag, \
iMHD_rho,iMHD_Ti,iMHD_Te, \
iMHD_Vx,iMHD_Vy,iMHD_Vz, \
idx_nb_part,idx_PIC_dt,idx_PIC_time = pic.Header()

idx_wrong_dt = idx_nb_part
idx_nb_wrong_dt = idx_PIC_dt

In [48]:
plt.rc('figure.subplot', top=0.94)
plt.rc('figure.subplot', bottom=0.16)
plt.rc('figure.subplot', left=0.14)
plt.rc('figure.subplot', right=0.92)
plt.figure(figsize=(14.,14.))

<Figure size 1008x1008 with 0 Axes>

<Figure size 1008x1008 with 0 Axes>

In [5]:
PIC_selected = np.asarray([])

selection_timing_start = time.time()
for i in range(PIC_iter_start, PIC_iter_stop):
    print "iter = ", i
    pic = PIC.PIC(workdir, prefix, i, savepath, Z, atomic_weight)
#
    nb_particles = pic.CountParticles_V2(idx_nb_part)
    print "count particles, done"    
#
    """Reading all the data for all the particles"""
    data_to_load = np.asarray([ipx, ipy, ipz, 
                               iPx, iPy, iPz, 
                               iBx, iBy, iBz, 
                               iEx, iEy, iEz, 
                               imass, icharge, 
                               idx_wrong_dt, 
                               idx_nb_wrong_dt,  # No.15
                               itag              # No.16
                              ])
    PICData = pic.ExtractAllData_V2(data_to_load, PIC_time_out_frequency, nb_particles, 15, 16, info=True)  #C> - indices 15 and 16 are for the nb of wrong dt of the particle AND for the tags
    print "extract all data, done"
#
    """Computing and plotting the energies of all the particles"""
    Energy = pic.ComputeEnergies(PICData[:,3:6],PICData[:,12], PICData[:,-1])
    print "compute energies for all the particles, done"
#
    """Creating a sub-category of particles based on a criterion (here: 10.*Eref --> energetic particles) and saving their tags
    WARNING: The function removes the particles selected from the initial PICData array"""
    #PICData, PICData_up = pic.DistinguishEnergy(PICData, Energy, 8.0e6) # 0.*Eref)         #C> - Here is a function using numpy.where function: it is very slow if the number of particles is large
    PICData_up = Energy[(Energy[:,0] > 4.5e4),:]                                            #C> - This way creates a mask and is way faster than the other.
    print "select the particles with energy higher than a certain value, here 4.5e4 [J]?"
#
    try :
        PIC_selected = np.append(PIC_selected, PICData_up[:,-1])
        print "Number of particles distinguished with the criterion: " + np.size(PICData_up[:,-1])
    except :
        pass

plt.close()
print '  *  '

iter =  0
count particles, done
Time: 0.00 ns, nb_particles = 5568682/5568682 with 0 fucked up particles removed.
extract all data, done


PICclass.py:406: RuntimeWarning: divide by zero encountered in divide
  ergs[:, 0] = 0.5*(datmass[:]*(c_light**2.0)*momenta[:])/(elementary_charge*((datmass[:]*c_light)**2 + momenta[:]))
PICclass.py:406: RuntimeWarning: invalid value encountered in divide
  ergs[:, 0] = 0.5*(datmass[:]*(c_light**2.0)*momenta[:])/(elementary_charge*((datmass[:]*c_light)**2 + momenta[:]))


compute energies for all the particles, done
select the particles with energy higher than a certain value, here 4.5e4 [J]?
iter =  1
count particles, done
Time: 2.00 ns, nb_particles = 3286402/3286402 with 0 fucked up particles removed.
extract all data, done
compute energies for all the particles, done
select the particles with energy higher than a certain value, here 4.5e4 [J]?
iter =  2
count particles, done
Time: 4.00 ns, nb_particles = 2708356/2708356 with 0 fucked up particles removed.
extract all data, done
compute energies for all the particles, done
select the particles with energy higher than a certain value, here 4.5e4 [J]?
iter =  3
count particles, done
Time: 6.00 ns, nb_particles = 2332640/2332640 with 0 fucked up particles removed.
extract all data, done
compute energies for all the particles, done
select the particles with energy higher than a certain value, here 4.5e4 [J]?
iter =  4
count particles, done
Time: 8.00 ns, nb_particles = 2020829/2020829 with 0 fucked up pa

In [6]:
"""Removes multiple appearance of the same particles"""
PIC_selected = np.unique(PIC_selected)
PIC_selected = PIC_selected[1:]
print "PIC first selection: ", np.size(PIC_selected), " particles selected"

PIC first selection:  1849960  particles selected


In [7]:
selection_timing_end = time.time()
print "All times swept and data read - selection of reduced sets of particles done | Time taken: " + str( "%.2f" % ((selection_timing_end-selection_timing_start)/60.) ) + " minutes"

All times swept and data read - selection of reduced sets of particles done | Time taken: 17.11 minutes


In [8]:
"""Takes 300 particles randomly in the array of the selected particles"""
random_list = random.sample(xrange(np.size(PIC_selected)), 300)
PIC_selection = np.sort(PIC_selected[random_list])

In [9]:
"""This is an array to save the quantities related to the particles for all the times"""
PICData_sub1 = np.zeros([np.size(PIC_selection),16,PIC_iter_stop+1])         #C> - The '16' is to be changed accordingly to the number of quantities taken
PICData_sub1[:,0,0] = PIC_selection

In [10]:
"""This is an array to save mainly the positions of the selected particles for all the times"""
BasicQ_sub1 = np.zeros([np.size(PIC_selection),14,PIC_iter_stop+1])
BasicQ_sub1[:,0,0] = PIC_selection

In [11]:
PIC_selection = PIC_selection.astype(int)

In [12]:
selection_timing_start = time.time()
for i in range(PIC_iter_start, PIC_iter_stop):
    pic = PIC.PIC(workdir, prefix, i, savepath, Z, atomic_weight)
#C> ----
    nb_particles = pic.CountParticles_V2(idx_nb_part)
    data_to_load = np.asarray([ipx,ipy,ipz,iPx,iPy,iPz,iBx,iBy,iBz,iEx,iEy,iEz,imass,icharge,iMHD_rho,iMHD_Ti,iMHD_Te,iMHD_Vx,iMHD_Vy,iMHD_Vz,idx_wrong_dt,idx_nb_wrong_dt,itag]) #  [ipx, ipy, ipz, iPx, iPy, iPz, iBx, iBy, iBz, iEx, iEy, iEz, imass, icharge, idx_wrong_dt, idx_nb_wrong_dt, itag])
    PICData = pic.ExtractAllData_V2(data_to_load, PIC_time_out_frequency, nb_particles, 21, 22)  #C> - indices 15 and 16 are for the nb of wrong dt of the particle AND for the tags
#C> ----
    max_tag = np.max(PICData[:,-1])
    up_size = np.size(PIC_selection[(PIC_selection < max_tag)])
    if (up_size > 0):
        PICData_first_set = PICData[PIC_selection[(PIC_selection < max_tag)],:]
        print "Time: " + str("%.2f" % (i*PIC_time_out_frequency)) + ", number of particles up found: " + str(np.shape(PICData_first_set))

        PICData_sub1[:up_size, 15, i+1] = PICData_first_set[:, -1]
        PICData_sub1[:up_size, 14, i+1] = PICData_first_set[:, 13]
        PICData_sub1[:up_size, 13, i+1] = PICData_first_set[:, 12]
        """Get and save the spatial coordinates, velocity and B-field components of the particles - for a later use to then study their trajectories and others"""
        BasicQ_sub1[:up_size, :, i+1] = PICData_first_set[:, :14]
        """Get the local (to the particle) MHD quantities: density (kg/m^3), ionic temperature(eV), electronic temperature (eV) and flow velocity (m/s)"""
####    PICData_sub1[:,7,1:] = 1.0e-4                                              #Vacuum of GORGON from 1e-5kg/m3 to 1e-4kg/m3
        PICData_sub1[:up_size, 7:10, i+1] = PICData_first_set[:, 14:17]
        PICData_sub1[:up_size, 10,   i+1] = np.sqrt(PICData_first_set[:, 17]**2. + PICData_first_set[:, 18]**2. + PICData_first_set[:, 19]**2.)
        """Get the local B- and E-fields (in this order!) moduli"""
        PICData_sub1[:up_size, 3, i+1] = np.linalg.norm(PICData_first_set[:, 6: 9], axis=1)
        PICData_sub1[:up_size, 4, i+1] = np.linalg.norm(PICData_first_set[:, 9:12], axis=1)
        """Get the perpendicular and parallel velocities (in this order!) a vector; the array for parallel velocities has four dimensions to save the sign with respect to B_local"""
        Vperp_first_set, Vpara_first_set = pic.V1_wrt_V2(PICData_first_set[:, 3:6], PICData_first_set[:, 6:9], PICData_sub1[:up_size, 0, 0])
        """Get the particle ion temperature (in eV) and then (in this order!) the square of the perpendicular velocity component modulus (in (m/s)^2)"""
        momenta_prime = np.linalg.norm(PICData_first_set[:,3:6], axis=1)*np.linalg.norm(PICData_first_set[:,3:6], axis=1)
        gamma_prime = np.sqrt(1. + (momenta_prime/(PICData_sub1[:up_size, 13, i+1]*c_light)**2))
        PICData_sub1[:up_size, 11, i+1] = (momenta_prime)/(elementary_charge*PICData_sub1[:up_size, 13, i+1]*gamma_prime**2)
        PICData_sub1[:up_size, 12, i+1] = (np.linalg.norm(Vperp_first_set[:, :3], axis=1)*np.linalg.norm(Vperp_first_set[:,:3], axis=1))/(PICData_sub1[:up_size, 13, i+1]*gamma_prime)**2
        """Get the perpendicular and parallel components (in this order!) of the electric field; the array for parallel electric field has four dimensions to save the sign with respect to B_local"""
        Efield_perp_first_set, Efield_para_first_set = pic.V1_wrt_V2(PICData_first_set[:, 9:12], PICData_first_set[:, 6:9], PICData_sub1[:up_size, 0, 0])
        """Save the perpendicular and parallel (in this order!) components of the electric field"""
        PICData_sub1[:up_size, 5, i+1] = np.linalg.norm(Efield_perp_first_set [:, :3],axis=1)
        PICData_sub1[:up_size, 6, i+1] = np.linalg.norm(Efield_para_first_set [:, :3],axis=1)
        """Get the energies: total, perpendicular and parallel (in this order!)"""
        PICData_sub1[:up_size, 0, i+1] = pic.ComputeEnergiesSelection(PICData_first_set[:, 3:6], PICData_sub1[:up_size, 13,i+1], PICData_sub1[:up_size, 15, i+1])[:,0]
        PICData_sub1[:up_size, 1, i+1], PICData_sub1[:up_size, 2, i+1] = pic.EpeEpa(Vperp_first_set, Vpara_first_set, PICData_sub1[:up_size, 13, i+1])
        print " ------------------ "
#C> - -----------------------------------------------------
    print "Time: " + str( "%.2f" % (i*PIC_time_out_frequency) ) + ", data read and quantities derived..."

selection_timing_end = time.time()
print "Reduced set of selected particles: data read and quantities derived... | Time taken: " + str( "%.2f" % ((selection_timing_end-selection_timing_start)/60.) ) + " minutes"
print "*"


Time: 0.00, number of particles up found: (300, 23)
 ------------------ 
Time: 0.00, data read and quantities derived...
Time: 2.00, number of particles up found: (300, 23)
 ------------------ 
Time: 2.00, data read and quantities derived...


PICclass.py:457: RuntimeWarning: invalid value encountered in divide
  tmp_Vector2[:,0] = Vector2[:,0]/V2_mag
PICclass.py:458: RuntimeWarning: invalid value encountered in divide
  tmp_Vector2[:,1] = Vector2[:,1]/V2_mag
PICclass.py:459: RuntimeWarning: invalid value encountered in divide
  tmp_Vector2[:,2] = Vector2[:,2]/V2_mag
/mesopsl3/home/wyao/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in divide
PICclass.py:374: RuntimeWarning: invalid value encountered in divide
  ergs[:,0] = 0.5*(datmass[:]*(c_light**2.0)*momenta[:])/(elementary_charge*((datmass[:]*c_light)**2 + momenta[:]))


Time: 4.00, number of particles up found: (300, 23)
 ------------------ 
Time: 4.00, data read and quantities derived...
Time: 6.00, number of particles up found: (300, 23)
 ------------------ 
Time: 6.00, data read and quantities derived...
Time: 8.00, number of particles up found: (300, 23)
 ------------------ 
Time: 8.00, data read and quantities derived...
Time: 10.00, number of particles up found: (300, 23)
 ------------------ 
Time: 10.00, data read and quantities derived...
Time: 12.00, number of particles up found: (300, 23)
 ------------------ 
Time: 12.00, data read and quantities derived...
Time: 14.00, number of particles up found: (300, 23)
 ------------------ 
Time: 14.00, data read and quantities derived...
Time: 16.00, number of particles up found: (300, 23)
 ------------------ 
Time: 16.00, data read and quantities derived...
Time: 18.00, number of particles up found: (300, 23)
 ------------------ 
Time: 18.00, data read and quantities derived...
Reduced set of selecte

In [13]:
selection_timing_start = time.time()


"""Conversion from momenta components to velocity components"""
BasicQ_sub1[:,3:6,1:] = pic.ConversionPj_to_Vj(BasicQ_sub1[:,3:6,1:])

In [14]:
"""Here the script is calling the PIC class deidcated to the plots (see PICclass.py script)"""
"""Plot of the spatial coordinates as function of time and as function of one another"""
picplot = PIC.PICPlot(workdir, nickname, savepath, 14., 14., .9, .1, .09, .96)
print "Calling for the function to plot the trajectories of the particles from the first subcategory"
pic.SecurityCheckPaths("Trajectories/Energetics/")
picplot.PlotTrajectories(PIC_iter_start, PIC_iter_stop, PIC_time_out_frequency, 8., 8., BasicQ_sub1[:,:3,:]*1e3, 50, "Trajectories/Energetics/", 'r-')   #(PIC_Time_start, PIC_Time_end, PIC_time_out_frequency, 8., 8., BasicQ_sub1[:,:3,:]*1e3, 20, "Trajectories/Energetics/", 'r-')

Calling for the function to plot the trajectories of the particles from the first subcategory
SecurityCheckPaths function: creation of the directory: /travail/wyao/data/PIC_SHOCK_GAS_test1/Treatment/Trajectories/Energetics/.
('Plot_trajectories - ', 0, ' images plotted')
('Plot_trajectories - ', 100, ' images plotted')
('Plot_trajectories - ', 200, ' images plotted')


In [21]:
""" Compute and plot some plasma parameters/quantities: cyclotron time, Larmor radius, first adiabatic invariant """
PlasmaQ_1 = np.zeros([np.size(PIC_selection),3,PIC_iter_stop])
PlasmaQ_1[:,0,:] = pic.ComputeTau_ci(PICData_sub1[:,3,1:], PICData_sub1[:,13,1:], PICData_sub1[:,14,1:])
PlasmaQ_1[:,1,:] = pic.ComputeLarmorRadius(PICData_sub1[:,11,1:], PICData_sub1[:,3,1:], PICData_sub1[:,13,1:], PICData_sub1[:,14,1:])
PlasmaQ_1[:,2,:] = pic.ComputeFirstAdiabaticInvariant(PICData_sub1[:,12,1:], PICData_sub1[:,3,1:], PICData_sub1[:,13,1:])

In [38]:
"""Here the script is calling the PIC class deidcated to the plots (see PICclass.py script)"""
picplot = PIC.PICPlot(workdir, nickname, savepath, 14., 14., .9, .1, .14, .96)
print "I'm going to plot the plasma quantities of the energetic particles"
pic.SecurityCheckPaths("Mu_Rho_and_Tau/Energetics/")
picplot.PlotPlasmaQuantities(PIC_iter_start, PIC_iter_stop, PIC_time_out_frequency, PICData_sub1[:,0,:], 1.0e3, PlasmaQ_1, 20, "Mu_Rho_and_Tau/Energetics/", 'b-')

I'm going to plot the plasma quantities of the energetic particles
('PlotPlasmaQuantities - ', 0, ' images plotted')
('PlotPlasmaQuantities - ', 100, ' images plotted')
('PlotPlasmaQuantities - ', 200, ' images plotted')
('Energy: average, standard deviation and std/average - ', 8.602814109222498, 15.565986429992893, ' and ', 1.8094063445245954)
('Larmor radius: average, std and std/average - ', 1.0108631258008161e+22, 5.150326466991126e+20, 'and ', 0.050949790684183725)
('First adiabatic invariant: avg, std and std/avg - ', 1.0197500586597028, 0.047376218178207315, 'and ', 0.04645865697764776)


In [20]:
np.size(PICData_sub1[:,1,1])

300

In [28]:
np.shape(PICData_sub1[1,0,0])

(300,)

In [29]:
np.shape(PlasmaQ_1)

(300, 3, 10)

In [34]:
np.shape(np.arange(PIC_iter_start,PIC_Time_end)*PIC_time_out_frequency)

(20,)

In [35]:
PIC_iter_start
PIC_iter_stop

10

In [32]:
PIC_Time_end

20.0

In [33]:
PIC_time_out_frequency

2.0